In [ ]:
# imports

import numpy as np
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from confusion_matrix import ConfusionPlots

In [ ]:
dir_path = '11_ds_2/'
class_names = np.load(dir_path+'class_names.npy')
orig_train_data = np.load(dir_path+'train_data.npy')
train_target = np.load(dir_path+'train_target.npy')
orig_test_data = np.load(dir_path+'test_data.npy')
test_target = np.load(dir_path+'test_target.npy')
print(np.shape(orig_train_data),np.shape(orig_test_data))
print(class_names)

In [ ]:
def min_max_norm(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))
print(np.max(orig_train_data))
train_data = orig_train_data
test_data = orig_test_data
print(np.max(train_data))

In [ ]:
show_data = np.reshape(train_data[0],(28,28))
plt.imshow(show_data,cmap='gray')
plt.show()

In [ ]:
# AANN1

dim = 784
h1_dim = 1000
h2_dim = 700
h3_dim = 600
h4_dim = 700
batch_size = 64
learning_rate = 0.01
num_epochs = 500
num_workers = 8

class AANN1(nn.Module):
    def __init__(self):
        super(AANN1, self).__init__()
        self.fc1 = nn.Linear(dim,h1_dim)
        self.fc2 = nn.Linear(h1_dim,h2_dim)
        self.fcl = nn.Linear(h2_dim, h3_dim)
        self.fc3 = nn.Linear(h3_dim,h4_dim)
        self.fc4 = nn.Linear(h4_dim,dim)
        
    def forward(self, x):
        m = nn.Softplus()
        x = self.fc1(x)
        x = m(self.fc2(x))
        y = self.fcl(x)
        x = m(self.fc3(y))
        x = self.fc4(x)
        return x, y

In [ ]:
tensor_data = torch.Tensor(train_data)
tensor_data_test = torch.Tensor(test_data)

dataset = torch.utils.data.TensorDataset(tensor_data,tensor_data)
dataset_test = torch.utils.data.TensorDataset(tensor_data_test,tensor_data_test)
loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,shuffle=True,num_workers=num_workers)
loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size,shuffle=True,num_workers=num_workers)


In [ ]:
# training for AANN1

# loading data

#aann1 = AANN1()
criterion = nn.MSELoss()
optimizer = optim.Adagrad(aann1.parameters(), lr=learning_rate,lr_decay=0)

# training
running_loss = 0.0
count = 0
for epoch in range(num_epochs):
    
    for i, load_data in enumerate(loader):
        
        inputs, same_inputs = load_data
        
        # initialize with zeros all param
        optimizer.zero_grad()
        
        outputs,_ = aann1(inputs)
        loss = criterion(outputs,same_inputs)
        loss.backward()
        optimizer.step()
        count += 1
        running_loss += loss.item()
        
    if((epoch+1)%50 == 0 or epoch == 0.0):
        print(epoch+1, running_loss/(count))
        running_loss = 0.0
        count = 0


In [ ]:
aann1 = AANN1()
aann1.load_state_dict(torch.load('aann1_task3.pth'))

In [ ]:
# computing error
error = 0
num = 0
with torch.no_grad():
    for data in loader:
        inputs, labels = data
        outputs,_ = aann1(inputs)
        diff = outputs-inputs
        error += ((diff*diff).sum().item())
        num += 1
print('Train error',end=':')
print(error/(num*dim))

error = 0
num = 0
with torch.no_grad():
    for data in loader_test:
        inputs, labels = data
        outputs,_ = aann1(inputs)
        diff = outputs-inputs
        error += ((diff*diff).sum().item())
        num += 1
print('Test error',end=':')
print(error/(num*dim))

In [ ]:
torch.save(aann1.state_dict(),'aann1_task3.pth')

In [ ]:
# AANN2

dim = 600 # from previous, AANN1, 600
h1_dim = 800
h2_dim = 600
h3_dim = 400
h4_dim = 600
batch_size = 64
learning_rate = 0.001
num_epochs = 500
num_workers = 8

class AANN2(nn.Module):
    def __init__(self):
        super(AANN2, self).__init__()
        self.fc1 = nn.Linear(dim,h1_dim)
        self.fc2 = nn.Linear(h1_dim,h2_dim)
        self.fcl = nn.Linear(h2_dim, h3_dim)
        self.fc3 = nn.Linear(h3_dim,h4_dim)
        self.fc4 = nn.Linear(h4_dim,dim)
        
    def forward(self, x):
        m = nn.Tanh()
        x = self.fc1(x)
        x = m(self.fc2(x))
        y = self.fcl(x)
        x = m(self.fc3(y))
        x = self.fc4(x)
        return x, y

In [ ]:
# training for AANN2

aann_train = torch.Tensor(train_data)
aann_test = torch.Tensor(test_data)

with torch.no_grad():
    print(aann_train.size())
    _, aann1_train = aann1(aann_train)
    _, aann1_test = aann1(aann_test)
    print(aann1_train.size())

# loading data

dataset = torch.utils.data.TensorDataset(aann1_train,aann1_train)
dataset_test = torch.utils.data.TensorDataset(aann1_test,aann1_test)
loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,shuffle=True,num_workers=num_workers)
loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size,shuffle=True,num_workers=num_workers)

#aann2 = AANN2()
criterion = nn.MSELoss()
optimizer = optim.Adagrad(aann2.parameters(), lr=learning_rate,lr_decay=0)

# training
running_loss = 0.0
count = 0
for epoch in range(num_epochs):
    
    for i, load_data in enumerate(loader):
        
        inputs, same_inputs = load_data
        
        # initialize with zeros all param
        optimizer.zero_grad()
        
        outputs,_ = aann2(inputs)
        loss = criterion(outputs,same_inputs)
        loss.backward()
        optimizer.step()
        count += 1
        running_loss += loss.item()
        
    if((epoch+1)%50 == 0):
        print(epoch+1, running_loss/count)
        running_loss = 0.0
        count = 0


In [ ]:
aannx = AANN()
aannx.load_state_dict(torch.load('aann2_task3.pth'))

In [ ]:
# computing error
error = 0
num = 0
with torch.no_grad():
    for data in loader:
        inputs, labels = data
        outputs,_ = aann2(inputs)
        diff = outputs-inputs
        error += ((diff*diff).sum().item())
        num += 1
print('Train error',end=':')
print(error/(num*dim))

error = 0
num = 0
with torch.no_grad():
    for data in loader_test:
        inputs, labels = data
        outputs,_ = aann2(inputs)
        diff = outputs-inputs
        error += ((diff*diff).sum().item())
        num += 1
print('Test error',end=':')
print(error/(num*dim))

In [ ]:
torch.save(aann2.state_dict(),'aann2_task3.pth')

In [ ]:
# AANN3

dim = 400 # from previous, AANN2, 500
h1_dim = 600
h2_dim = 400
h3_dim = 300
h4_dim = 350
batch_size = 64
learning_rate = 0.001
num_epochs = 1000
num_workers = 8

class AANN3(nn.Module):
    def __init__(self):
        super(AANN3, self).__init__()
        self.fc1 = nn.Linear(dim,h1_dim)
        self.fc2 = nn.Linear(h1_dim,h2_dim)
        self.fcl = nn.Linear(h2_dim, h3_dim)
        self.fc3 = nn.Linear(h3_dim,h4_dim)
        self.fc4 = nn.Linear(h4_dim,dim)
        
    def forward(self, x):
        m = nn.Tanh()
        x = self.fc1(x)
        x = m(self.fc2(x))
        y = self.fcl(x)
        x = m(self.fc3(y))
        x = self.fc4(x)
        return x, y

In [ ]:
# training for AANN3

# loading data
with torch.no_grad():
    print(aann1_train.size())
    _, aann2_train = aann2(aann1_train)
    _, aann2_test = aann2(aann1_test)
    print(aann2_train.size())

dataset = torch.utils.data.TensorDataset(aann2_train,aann2_train)
dataset_test = torch.utils.data.TensorDataset(aann2_test,aann2_test)
loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,shuffle=True,num_workers=num_workers)
loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size,shuffle=True,num_workers=num_workers)

aann3 = AANN3()
criterion = nn.MSELoss()
optimizer = optim.Adagrad(aann3.parameters(), lr=learning_rate,lr_decay=0)

# training
running_loss = 0.0
count = 0
for epoch in range(num_epochs):
    
    for i, load_data in enumerate(loader):
        
        inputs, same_inputs = load_data
        
        # initialize with zeros all param
        optimizer.zero_grad()
        
        outputs,_ = aann3(inputs)
        loss = criterion(outputs,same_inputs)
        loss.backward()
        optimizer.step()
        count += 1
        running_loss += loss.item()
        
    if((epoch+1)%50 == 0):
        print(epoch+1, running_loss/count)
        running_loss = 0.0
        count = 0


In [ ]:
aannx = AANN3()
aannx.load_state_dict(torch.load('aann3_task3.pth'))

In [ ]:
# computing error
error = 0
num = 0
with torch.no_grad():
    for data in loader:
        inputs, labels = data
        outputs,_ = aann3(inputs)
        diff = outputs-inputs
        error += ((diff*diff).sum().item())
        num += 1
print('Train error',end=':')
print(error/(num*dim))

error = 0
num = 0
with torch.no_grad():
    for data in loader_test:
        inputs, labels = data
        outputs,_ = aann3(inputs)
        diff = outputs-inputs
        error += ((diff*diff).sum().item())
        num += 1
print('Test error',end=':')
print(error/(num*dim))

In [ ]:
torch.save(aann3.state_dict(),'aann3_task3.pth')

In [ ]:
aann_train = torch.Tensor(train_data)
aann_test = torch.Tensor(test_data)

with torch.no_grad():
    print(aann_train.size())
    _, aann_train = aann1(aann_train)
    _, aann_train = aann2(aann_train)
    _, aann_train = aann3(aann_train)
    _, aann_test = aann1(aann_test)
    _, aann_test = aann2(aann_test)
    _, aann_test = aann3(aann_test)
    print(aann_train.size())

In [ ]:
# parameters

dim = 300 # from output of aann2, 300
h1_dim = 50
h2_dim = 28
out_dim = len(class_names)
learning_rate = 0.001
momentum = 0.9
num_epochs = 1000
batch_size = 32  # general rule is to try 32, 64, 128 ... 

# DNN model with 2 fully connected hidden layers

class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(dim,h1_dim) 
        self.fc2 = nn.Linear(h1_dim,h2_dim) 
        self.fc3 = nn.Linear(h2_dim,out_dim)
        
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# loss fun and update method

tensor_train_target = torch.LongTensor(train_target) # long otherwise takes as float

tensor_test_target = torch.LongTensor(test_target)

# num_workers - how many parallel data loadings
train_dataset = torch.utils.data.TensorDataset(aann_train,tensor_train_target)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,shuffle=True,num_workers=8)

test_dataset = torch.utils.data.TensorDataset(aann_test,tensor_test_target)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,shuffle=False,num_workers=8)

net = DNN()
criterion = nn.CrossEntropyLoss()
# ASDG for generalised delta ig
# schochastic gradient descent with nestrov momentum
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)
# optimizer = optim.Adagrad(net.parameters(), lr=learning_rate,lr_decay=0)

In [ ]:
# training


running_loss = 0.0
count = 0
for epoch in range(num_epochs):
    
    for i, load_data in enumerate(train_loader):
        
        inputs, labels = load_data
        
        # initialize with zeros all param
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        count += 1
        running_loss += loss.item()
        
    if((epoch+1)%50 == 0):
        print(epoch+1, running_loss/count)
        running_loss = 0.0
        count = 0

In [ ]:
net = DNN()
net.load_state_dict(torch.load('net_task3.pth'))

In [ ]:
# testing

# train data
correct = 0
total = 0
preds = []
tgts = []
with torch.no_grad():
    for data in train_loader:
        inputs, labels = data
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        preds.extend(predicted)
        tgts.extend(labels)
print('train accuracy',end=':')
print(correct/total)

ConfusionPlots.pretty_conf(tgts,preds,'Task3-TrainData-3-Stacked-AANN',columns=class_names)

#test data
correct = 0
total = 0
preds = []
tgts = []
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        preds.extend(predicted)
        tgts.extend(labels)
print('test accuracy',end=':')
print(correct/total)

ConfusionPlots.pretty_conf(tgts,preds,'Task3-TestData-3-Stacked-AANN',columns=class_names)

In [ ]:
torch.save(net.state_dict(),'net_task3.pth')